In [ ]:
import argparse
import os
import numpy as np
import trimesh
from matplotlib import pyplot as plt
from PIL import Image


# Loading utilities
def load_objects(obj_root):
    object_names = ['juice_bottle', 'liquid_soap', 'milk', 'salt']
    all_models = {}
    for obj_name in object_names:
        obj_path = os.path.join(obj_root, '{}_model'.format(obj_name),
                                '{}_model.ply'.format(obj_name))
        mesh = trimesh.load(obj_path)
        all_models[obj_name] = {
            'verts': np.array(mesh.vertices),
            'faces': np.array(mesh.faces)
        }
    return all_models


def get_skeleton(sample, skel_root):
    skeleton_path = os.path.join(skel_root, sample['subject'],
                                 sample['action_name'], sample['seq_idx'],
                                 'skeleton.txt')
    print('Loading skeleton from {}'.format(skeleton_path))
    skeleton_vals = np.loadtxt(skeleton_path)
    skeleton = skeleton_vals[:, 1:].reshape(skeleton_vals.shape[0], 21,
                                            -1)[sample['frame_idx']]
    return skeleton


def get_obj_transform(sample, obj_root):
    seq_path = os.path.join(obj_root, sample['subject'], sample['action_name'],
                            sample['seq_idx'], 'object_pose.txt')
    with open(seq_path, 'r') as seq_f:
        raw_lines = seq_f.readlines()
    raw_line = raw_lines[sample['frame_idx']]
    line = raw_line.strip().split(' ')
    trans_matrix = np.array(line[1:]).astype(np.float32)
    trans_matrix = trans_matrix.reshape(4, 4).transpose()
    print('Loading obj transform from {}'.format(seq_path))
    return trans_matrix


# Display utilities
def visualize_joints_2d(ax, joints, joint_idxs=True, links=None, alpha=1):
    """Draw 2d skeleton on matplotlib axis"""
    if links is None:
        links = [(0, 1, 2, 3, 4), (0, 5, 6, 7, 8), (0, 9, 10, 11, 12),
                 (0, 13, 14, 15, 16), (0, 17, 18, 19, 20)]
    # Scatter hand joints on image
    x = joints[:, 0]
    y = joints[:, 1]
    ax.scatter(x, y, 1, 'r')

    # Add idx labels to joints
    for row_idx, row in enumerate(joints):
        if joint_idxs:
            plt.annotate(str(row_idx), (row[0], row[1]))
    _draw2djoints(ax, joints, links, alpha=alpha)


def _draw2djoints(ax, annots, links, alpha=1):
    """Draw segments, one color per link"""
    colors = ['r', 'm', 'b', 'c', 'g']

    for finger_idx, finger_links in enumerate(links):
        for idx in range(len(finger_links) - 1):
            _draw2dseg(
                ax,
                annots,
                finger_links[idx],
                finger_links[idx + 1],
                c=colors[finger_idx],
                alpha=alpha)


def _draw2dseg(ax, annot, idx1, idx2, c='r', alpha=1):
    """Draw segment of given color"""
    ax.plot(
        [annot[idx1, 0], annot[idx2, 0]], [annot[idx1, 1], annot[idx2, 1]],
        c=c,
        alpha=alpha)
    
def get_data_list(modality, dataset_dir):
    train_pairs = []
    test_pairs = []
    img_dir = os.path.join(dataset_dir, 'Video_files')
    skel_dir = os.path.join(dataset_dir, 'Hand_pose_annotation_v1')
    if modality == 'depth':
        img_type = 'png'
    else:
        img_type = 'jpeg'
    with open(os.path.join(dataset_dir, 'data_split_action_recognition.txt')) as f:
        cur_split = 'Training'
        lines = f.readlines()
        for l in lines:
            words = l.split()
            if(words[0] == 'Training' or words[0] == 'Test'):
                cur_split = words[0]
            else:
                path = l.split()[0]
                full_path = os.path.join(img_dir, path, modality)
                len_frame_idx = len([x for x in os.listdir(full_path)
                                    if os.path.join(full_path, x)])
                skeleton_path = os.path.join(skel_dir, path, 'skeleton.txt')
                skeleton_vals = np.loadtxt(skeleton_path)
                for i in range(len_frame_idx):
                    img_path = os.path.join(img_dir, path, modality, '%s_%04d.%s' %(modality, i, img_type))
                    skel_xyz = skeleton_vals[:, 1:].reshape(skeleton_vals.shape[0], -1)[i]
                    data_pair = (img_path, skel_xyz)
                    if cur_split == 'Training':
                        train_pairs.append(data_pair)
                    else:
                        test_pairs.append(data_pair)
    return train_pairs, test_pairs

In [ ]:
root = '/4TB/aaron/First_Person_Action_Benchmark'
train_pairs, test_pairs = get_data_list('color', root)
file_name = [i for i,j in train_pairs]
train_file_name = [file.split('/') for file in file_name]
file_name = [i for i,j in test_pairs]
test_file_name = [file.split('/') for file in file_name]


In [ ]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--root', required=True, help='Path to dataset install')
# parser.add_argument('--subject', required=True, default='Subject_1')
# parser.add_argument('--action_name', required=True, default='open_liquid_soap')
# parser.add_argument('--seq_idx', required=True, default='1')
# parser.add_argument('--frame_idx', required=True, default=0, type=int)
# parser.add_argument(
#     '--obj', required=True, choices=['liquid_soap', 'juice_bottle', 'milk', 'salt'])
# args = parser.parse_args()
reorder_idx = np.array([
    0, 1, 6, 7, 8, 2, 9, 10, 11, 3, 12, 13, 14, 4, 15, 16, 17, 5, 18, 19,
    20
])

file_name = train_file_name
print(len(file_name))


# root = '/4TB/aaron/First_Person_Action_Benchmark'
# subject = 'Subject_1'
# action_name = 'open_liquid_soap'
# seq_idx = '1'
# frame_idx = 0
idx = 34110                 
subject = file_name[idx][5]
action_name = file_name[idx][6]
seq_idx = file_name[idx][7]
frame_idx = int(file_name[idx][-1][-9:-5])

# obj = 'liquid_soap'
obj = None
sample = {
    'subject': subject,
    'action_name': action_name,
    'seq_idx': seq_idx,
    'frame_idx': frame_idx,
    'object': obj
}

print('Loading sample {}'.format(sample))
cam_extr = np.array(
    [[0.999988496304, -0.00468848412856, 0.000982563360594,
      25.7], [0.00469115935266, 0.999985218048, -0.00273845880292, 1.22],
     [-0.000969709653873, 0.00274303671904, 0.99999576807,
      3.902], [0, 0, 0, 1]])
cam_intr = np.array([[1395.749023, 0, 935.732544],
                     [0, 1395.749268, 540.681030], [0, 0, 1]])
cam_intr_depth = [[475.065948, 0,  315.944855],
                  [0, 475.065857, 245.287079],
                  [0, 0, 1]]


skeleton_root = os.path.join(root, 'Hand_pose_annotation_v1')
obj_root = os.path.join(root, 'Object_models')
obj_trans_root = os.path.join(root, 'Object_6D_pose_annotation_v1')
skel = get_skeleton(sample, skeleton_root)[reorder_idx]

# Apply camera extrinsic to hand skeleton
skel_hom = np.concatenate([skel, np.ones([skel.shape[0], 1])], 1)
skel_camcoords = cam_extr.dot(
    skel_hom.transpose()).transpose()[:, :3].astype(np.float32)

skel_hom2d = np.array(cam_intr).dot(skel_camcoords.transpose()).transpose()
skel_proj = (skel_hom2d / skel_hom2d[:, 2:])[:, :2]

# Plot everything
# Load image and display
# ax = fig.add_subplot(221)
img_path = os.path.join(root, 'Video_files', sample['subject'],
                        sample['action_name'], sample['seq_idx'], 'color',
                        'color_{:04d}.jpeg'.format(sample['frame_idx']))
print('Loading image from {}'.format(img_path))
img = Image.open(img_path)
# ax.imshow(img)

fig, ax = plt.subplots(1, 2, figsize=(15,15))
ax[0].imshow(img)
visualize_joints_2d(ax[0], skel_proj, joint_idxs=False)

skel_hom2d = np.array(cam_intr_depth).dot(skel.transpose()).transpose()
skel_proj = (skel_hom2d / skel_hom2d[:, 2:])[:, :2]

# Plot everything
# Load image and display
# ax = fig.add_subplot(221)
img_path = os.path.join(root, 'Video_files', sample['subject'],
                        sample['action_name'], sample['seq_idx'], 'depth',
                        'depth_{:04d}.png'.format(sample['frame_idx']))
print('Loading image from {}'.format(img_path))
img = Image.open(img_path)
# ax.imshow(img)

ax[1].imshow(img)
visualize_joints_2d(ax[1], skel_proj, joint_idxs=False)